In [1]:
%%capture
pip install plotly pandas statsmodels kaleido scipy nbformat jinja2

In [2]:
# read CSV data

import glob
import re
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import numpy as np
import os.path
import pickle
import scipy
from statistics import mean, stdev
from math import sqrt, log10
from packaging.version import Version

output_directory = '../output-linux-2024-11-08'
figures_directory = '../../paper-splc-2025-time-travel/images'
default_height = 270

pio.templates['colorblind'] = go.layout.Template(layout_colorway=['#648FFF', '#FE6100', '#785EF0', '#DC267F', '#FFB000'])
pio.templates.default = 'plotly_white+colorblind'

def read_dataframe(stage, dtype={}, usecols=None, file=None):
    if not file:
        file = 'output'
    df = pd.read_csv(f'{output_directory}/{stage}/{file}.csv', dtype=dtype, usecols=usecols)
    if 'committer_date_unix' in df:
        df['committer_date'] = df['committer_date_unix'].apply(lambda d: pd.to_datetime(d, unit='s'))
    return df

def replace_values(df):
    df.replace('kconfigreader', 'KConfigReader', inplace=True)
    df.replace('kmax', 'KClause', inplace=True)

def big_log10(str):
    return log10(int(str)) if not pd.isna(str) and str != '' else pd.NA

def process_model_count(df_solve):
    df_solve['model-count'] = df_solve['model-count'].replace('1', '')
    df_solve['model-count-log10'] = df_solve['model-count'].fillna('').apply(big_log10).replace(0, np.nan)
    df_solve['year'] = df_solve['committer_date'].apply(lambda d: int(d.year))

def peek_dataframe(df, column, message, type='str', filter=['revision', 'architecture', 'extractor']):
    success = df[~df[column].str.contains('NA') if type == 'str' else ~df[column].isna()][filter]
    failure = df[df[column].str.contains('NA') if type == 'str' else df[column].isna()][filter]
    print(f'{message}: {len(success)} successes, {len(failure)} failures')

df_architectures = read_dataframe(f'read-linux-architectures')
df_architectures = df_architectures.sort_values(by='committer_date')
df_architectures['year'] = df_architectures['committer_date'].apply(lambda d: int(d.year))

df_configs = read_dataframe(f'read-linux-configs')
df_configs = df_configs[~df_configs['kconfig-file'].str.contains('/um/')]

df_config_types = read_dataframe(f'read-linux-configs', file='output.types')
df_config_types = df_config_types[~df_config_types['kconfig-file'].str.contains('/um/')]
df_config_types = df_config_types.merge(df_architectures[['revision', 'committer_date']].drop_duplicates())

df_kconfig = read_dataframe('kconfig')
df_kconfig['year'] = df_kconfig['committer_date'].apply(lambda d: int(d.year))

df_uvl = read_dataframe('model_to_uvl_featureide')
df_smt = read_dataframe('model_to_smt_z3')
df_dimacs = read_dataframe('dimacs')
df_backbone_dimacs = read_dataframe('backbone-dimacs')

df_solve = read_dataframe('solve_model-count', {'model-count': 'string'})
process_model_count(df_solve)

if os.path.isfile(f'{output_directory}/model-count-with-6h-timeout.csv'):
    df_solve_6h = pd.read_csv(f'{output_directory}/model-count-with-6h-timeout.csv', dtype={'model-count': 'string'})
    df_solve_6h = df_backbone_dimacs.merge(df_solve_6h)
    process_model_count(df_solve_6h)
    df_solve = pd.merge(df_solve, df_solve_6h[['revision','architecture', 'extractor', 'backbone.dimacs-analyzer']], indicator=True, how='outer') \
        .query('_merge=="left_only"') \
        .drop('_merge', axis=1)
    df_solve = pd.concat([df_solve, df_solve_6h])
else:
    df_solve_6h = None

for df in [df_kconfig, df_uvl, df_smt, df_dimacs, df_backbone_dimacs, df_solve]:
    replace_values(df)

peek_dataframe(df_kconfig, 'model-file', 'model extraction')
peek_dataframe(df_uvl, 'uvl-file', 'UVL transformation', 'na', ['model-file'])
peek_dataframe(df_smt, 'smt-file', 'SMT transformation', 'na', ['model-file'])
peek_dataframe(df_dimacs, 'dimacs-file', 'CNF transformation')
peek_dataframe(df_backbone_dimacs, 'backbone.dimacs-file', 'backbone transformation', 'na')
peek_dataframe(df_solve, 'model-count-log10', 'model counting', 'na')

model extraction: 6572 successes, 46 failures
UVL transformation: 6572 successes, 0 failures
SMT transformation: 6572 successes, 0 failures
CNF transformation: 6572 successes, 0 failures
backbone transformation: 6496 successes, 76 failures
model counting: 4260 successes, 8732 failures


In [3]:
# helper functions for drawing plots

def estimate_group(group):
    print('\\hspace{2mm} ' + group + ' \\\\')

def estimate_trend(fig, color=None, color_value=None, xs=[], key=lambda x: x.timestamp()):
    results = px.get_trendline_results(fig)
    if color is not None and color_value is not None:
        idx = [i for i, r in enumerate(results.iloc) if r[color] == color_value][0]
    else:
        idx = 0
    intercept = results.iloc[idx]['px_fit_results'].params[0]
    slope = results.iloc[idx]['px_fit_results'].params[1]
    daily = slope * pd.to_timedelta(1, unit='D').total_seconds()
    weekly = slope * pd.to_timedelta(7, unit='D').total_seconds()
    monthly = slope * pd.to_timedelta(1, unit='D').total_seconds() * 30.437
    yearly = slope * pd.to_timedelta(1, unit='D').total_seconds() * 365.25
    return daily, weekly, monthly, yearly, [intercept + slope * key(x) for x in xs]

def committer_date_x_axis(fig, df=df_kconfig, append_revision=True, step=1):
    axis = df_kconfig[['committer_date', 'revision']].drop_duplicates()
    axis['year'] = axis['committer_date'].apply(lambda d: str(d.year))
    axis = axis.sort_values(by='committer_date').groupby('year').nth(0).reset_index()
    fig.update_xaxes(
        ticktext=axis['year'].str.cat('<br><sup>' + axis['revision'].str[1:] + '</sup>')[1::step] if append_revision else axis['year'][::step],
        tickvals=axis['year'][1::step]
    )

def revision_x_axis(fig, df=df_kconfig):
    axis = df_kconfig[['committer_date', 'revision']].drop_duplicates()
    axis['year'] = axis['committer_date'].apply(lambda d: str(d.year))
    axis = axis.sort_values(by='committer_date').groupby('year').nth(0).reset_index()
    fig.update_xaxes(
        ticktext=axis['year'],
        tickvals=axis['revision']
    )

def log10_y_axis(fig):
    fig.update_yaxes(tickprefix = "10<sup>", ticksuffix = "</sup>")

def percentage_y_axis(fig):
    fig.layout.yaxis.tickformat = ',.0%'

def format_percentage(value):
    return str(round(value * 100, 2)) + '%'

def committer_date_labels(dict={}):
    return {'committer_date': 'Year<br><sup>First Release in Year</sup>'} | dict

def revision_labels(dict={}):
    return {'revision': 'Year'} | dict

def style_legend(fig, position='topleft', xshift=0, yshift=0):
    if position == 'topleft':
        fig.update_layout(legend=dict(yanchor='top', y=0.98 + yshift, xanchor='left', x=0.01 + xshift))
    elif position == 'topright':
        fig.update_layout(legend=dict(yanchor='top', y=0.98 + yshift, xanchor='right', x=0.98 + xshift))
    elif position == 'bottomright':
        fig.update_layout(legend=dict(yanchor='bottom', y=0.01 + yshift, xanchor='right', x=0.98 + xshift))
    elif position == 'bottomleft':
        fig.update_layout(legend=dict(yanchor='bottom', y=0.01 + yshift, xanchor='left', x=0.01 + xshift))
    else:
        fig.update_layout(showlegend=False)

def style_box(fig, legend_position='topleft', xshift=0, yshift=0):
    fig.update_traces(fillcolor='rgba(0,0,0,0)')
    fig.update_traces(line_width=1)
    fig.update_traces(marker_size=2)
    fig.update_layout(font_family="Linux Biolinum")
    style_legend(fig, legend_position, xshift, yshift)

def style_scatter(fig, marker_size=4, legend_position='topleft', xshift=0, yshift=0):
    if marker_size:
        fig.update_traces(marker_size=marker_size)
    style_legend(fig, legend_position, xshift, yshift)
    fig.update_layout(font_family="Linux Biolinum")

def plot_failures(fig, df, x, y, y_value, align='bottom', xref='x', font_size=10, textangle=270):
    group = df.groupby(x, dropna=False)
    failures = (group[y].size() - group[y].count()).reset_index().rename(columns={y: f'{y}_failures'})
    attempts = group[y].size().reset_index().rename(columns={y: f'{y}_attempts'})
    failures = pd.merge(failures, attempts)
    failures[f'{y}_text'] = failures[f'{y}_failures'].astype(str) + ' (' + (failures[f'{y}_failures'] / failures[f'{y}_attempts']).apply(lambda v: "{0:.1f}%".format(v * 100)) + ')'
    for row in range(len(failures)):
        text = failures.at[row, f'{y}_text']
        text = "" if failures.at[row, f'{y}_failures'] == 0 else text
        fig.add_annotation(
            x=failures.at[row, x],
            y=y_value,
            text=text,
            showarrow=False,
            font_size=font_size,
            textangle=textangle,
            align='left' if align == 'bottom' else 'right',
            yanchor='bottom' if align == 'bottom' else 'top',
            yshift=5 if align == 'bottom' else -5,
            font_color='gray',
            xref=xref
        )

def cohens_d(d1, d2):
    # uses pooled standard deviation
    n1, n2 = len(d1), len(d2)
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    u1, u2 = np.mean(d1), np.mean(d2)
    return (u1 - u2) / s

def wilcoxon_test(df, column_a, column_b):
    # if the same values are returned for many inputs, refer to https://stats.stackexchange.com/q/232927
    a = df[column_a][~df[column_a].isna()]
    b = df[column_b][~df[column_b].isna()]
    d = a - b
    results = scipy.stats.wilcoxon(d, method='approx')
    p = results.pvalue
    # adapted from https://stats.stackexchange.com/q/133077
    r = np.abs(results.zstatistic / np.sqrt(len(d) * 2))
    return p, r

def style_p_values(fig, brackets, scale=0, _format=dict(interline=0.07, text_height=1.07, color='gray')):
    # adapted from https://stackoverflow.com/q/67505252
    for entry in brackets:
        first_column, second_column, y, results = entry
        y_range = [1.01+y*_format['interline'], 1.02+y*_format['interline']]
        p, r = results
        if p >= 0.05:
            symbol = 'ns'
        elif p >= 0.01: 
            symbol = '*'
        elif p >= 0.001:
            symbol = '**'
        else:
            symbol = '***'
        first_column = first_column - scale
        second_column = second_column + scale
        fig.add_shape(type="line",
            xref="x", yref="y domain",
            x0=first_column, y0=y_range[0],
            x1=first_column, y1=y_range[1],
            line=dict(color=_format['color'], width=2,)
        )
        fig.add_shape(type="line",
            xref="x", yref="y domain",
            x0=first_column, y0=y_range[1], 
            x1=second_column, y1=y_range[1],
            line=dict(color=_format['color'], width=2,)
        )
        fig.add_shape(type="line",
            xref="x", yref="y domain",
            x0=second_column, y0=y_range[0], 
            x1=second_column, y1=y_range[1],
            line=dict(color=_format['color'], width=2,)
        )
        fig.add_annotation(dict(font=dict(color=_format['color'],size=14),
            x=(first_column + second_column)/2,
            y=y_range[1]*_format['text_height'],
            showarrow=False,
            text=symbol + ' <sup>(' + str(round(r, 2)) + ')</sup>',
            textangle=0,
            xref="x",
            yref="y domain"
        ))
    return fig

def bracket_for(i, j, xshift, y, results):
    return [i + xshift, j + xshift, y, results]

def filter_extractor(df, extractor):
    return df[df['extractor'] == extractor]

def annotate_value(fig, x, y, subplot, prefix, ax, ay, xanchor, df, fn=lambda prefix, y: prefix + ': ' + format(round(y), ',') if y > 0 else prefix):
    if isinstance(x, str):
        x = df[x].iat[0]
    if isinstance(y, str):
        y = df[y].iat[0]
    fig.add_annotation(
        xref='x' + str(subplot),
        yref='y' + str(subplot),
        x=x,
        y=y,
        ax=ax,
        ay=ay,
        xanchor=xanchor,
        text=fn(prefix, y)
    )

def show(fig, name=None, width=1000, height=500, margin=None):
    fig.update_layout(width=width, height=height)
    if margin:
        fig.update_layout(margin=margin)
    else:
        fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
    if figures_directory and os.path.isdir(figures_directory) and name:
        fig.write_image(f'{figures_directory}/{name}.pdf')
    fig.show()

In [4]:
# differentiate kinds of features

potential_misses_grep = set()
potential_misses_kmax = set()
extractor_comparison = {}
df_configs_configurable = df_configs.copy()
df_configs_configurable['configurable'] = False

def jaccard(a, b):
    return len(set.intersection(a, b)) / len(set.union(a, b))

def add_features(descriptor, source, features, min=2):
    descriptor[f'#{source}'] = len(features) if features is not None and len(features) >= min else np.nan

def get_variables(variable_map):
    variables = set(variable_map.values())
    if len(variables) <= 1:
        variables = set()
    return variables

def read_unconstrained_feature_variables(extractor, revision, architecture):
    unconstrained_features_filename = f'{output_directory}/unconstrained-features/{extractor}/linux/{revision}[{architecture}].unconstrained.features'
    unconstrained_feature_variables = set()
    if os.path.isfile(unconstrained_features_filename):
        with open(unconstrained_features_filename, 'r') as f:
            unconstrained_feature_variables = set([re.sub('^CONFIG_', '', f.strip()) for f in f.readlines()])
    return unconstrained_feature_variables

def inspect_architecture_features_for_model(extractor, revision, architecture, config_features, features_for_last_revision):
    global potential_misses_grep, potential_misses_kmax
    
    features_filename = f'{output_directory}/kconfig/{extractor}/linux/{revision}[{architecture}].features'
    with open(features_filename, 'r') as f:
        extracted_features = set([re.sub('^CONFIG_', '', f.strip()) for f in f.readlines()])
    
    unconstrained_feature_variables = read_unconstrained_feature_variables(extractor, revision, architecture)

    dimacs_filename = f'{output_directory}/backbone-dimacs/{extractor}/linux/{revision}[{architecture}].backbone.dimacs'
    all_variables = set()
    variables = set()
    feature_variables = set()
    core_feature_variables = set()
    dead_feature_variables = set()
    undead_feature_variables = set()
    all_feature_variables = set()
    features = set()
    core_features = set()
    unconstrained_features = set()
    constrained_features = set()
    added_features = None
    removed_features = None
    infos = {'extracted_features_jaccard': np.nan, \
                     'all_variables_jaccard': np.nan, \
                     'variables_jaccard': np.nan, \
                     'feature_variables_jaccard': np.nan, \
                     'undead_feature_variables_jaccard': np.nan, \
                     'all_feature_variables_jaccard': np.nan, \
                     'features_jaccard': np.nan, \
                     'unconstrained_bools': np.nan, \
                     'unconstrained_tristates': np.nan}
    
    if os.path.isfile(dimacs_filename):
        with open(dimacs_filename, 'r') as f:
            lines = f.readlines()
            all_variable_map = {}
            variable_map = {}
            feature_variable_map = {}
            for f in lines:
                if f.startswith('c '):
                    result = re.search('^c ([^ ]+) ([^ ]+)$', f)
                    if result:
                        index = int(result.group(1).strip())
                        name = result.group(2).strip()
                        all_variable_map[index] = name
                        if "k!" not in name:
                            variable_map[index] = name
                            if name != 'True' \
                                and name != '<unsupported>' \
                                and name != 'PREDICATE_Compare' \
                                and not name.startswith('__VISIBILITY__CONFIG_') \
                                and not name.endswith('_MODULE'):
                                feature_variable_map[index] = name
            all_variables = get_variables(all_variable_map)
            variables = get_variables(variable_map)
            feature_variables = get_variables(feature_variable_map)

            backbone_features_filename = f'{output_directory}/backbone-features/{extractor}/linux/{revision}[{architecture}].backbone.features'
            if os.path.isfile(backbone_features_filename):
                with open(backbone_features_filename, 'r') as f:
                    lines = f.readlines()
                    if len(lines) > 1:
                        core_feature_variables = set([line[1:].strip() for line in lines if line.startswith('+')]).intersection(feature_variables)
                        dead_feature_variables = set([line[1:].strip() for line in lines if line.startswith('-')]).intersection(feature_variables)

            if len(feature_variables) > 0:
                undead_feature_variables = feature_variables.difference(dead_feature_variables)
                all_feature_variables = undead_feature_variables.union(unconstrained_feature_variables)
                features = all_feature_variables.intersection(config_features)
                if f'{revision}###{architecture}' not in extractor_comparison:
                    extractor_comparison[f'{revision}###{architecture}'] = features
                else:
                    extractor_comparison[f'{revision}###{architecture}'] = jaccard(extractor_comparison[f'{revision}###{architecture}'], features)
                core_features = features.intersection(core_feature_variables)
                unconstrained_features = features.intersection(unconstrained_feature_variables)
                unconstrained_features_by_type = pd.DataFrame(list(unconstrained_features), columns=['config']) \
                    .merge(df_config_types[(df_config_types['revision'] == revision)])
                unconstrained_bools = unconstrained_features_by_type[unconstrained_features_by_type['type'] == 'bool']['config'].drop_duplicates()
                unconstrained_tristates = unconstrained_features_by_type[unconstrained_features_by_type['type'] == 'tristate']['config'].drop_duplicates()
                constrained_features = features.difference(core_feature_variables).difference(unconstrained_feature_variables)
                if architecture in features_for_last_revision and len(features_for_last_revision[architecture]) > 0:
                    added_features = features.difference(features_for_last_revision[architecture])
                    removed_features = features_for_last_revision[architecture].difference(features)
                infos = { \
                            'extracted_features_jaccard': jaccard(extracted_features, features), \
                            'all_variables_jaccard': jaccard(all_variables, features), \
                            'variables_jaccard': jaccard(variables, features), \
                            'feature_variables_jaccard': jaccard(feature_variables, features), \
                            'undead_feature_variables_jaccard': jaccard(undead_feature_variables, features), \
                            'all_feature_variables_jaccard': jaccard(all_feature_variables, features), \
                            'features_jaccard': 1, \
                            'unconstrained_bools': len(unconstrained_bools), \
                            'unconstrained_tristates': len(unconstrained_tristates) \
                        }
    descriptor = {'extractor': extractor, 'revision': revision, 'architecture': architecture} | infos
    add_features(descriptor, 'config_features', config_features) # F_config
    add_features(descriptor, 'extracted_features', extracted_features) # F_extracted
    add_features(descriptor, 'unconstrained_feature_variables', unconstrained_feature_variables, min=1) # F_unconstrained
    add_features(descriptor, 'all_variables', all_variables) # V_all
    add_features(descriptor, 'variables', variables) # V_phi
    add_features(descriptor, 'feature_variables', feature_variables) # FV_phi
    add_features(descriptor, 'core_feature_variables', core_feature_variables, min=1) # FV_core
    add_features(descriptor, 'dead_feature_variables', dead_feature_variables, min=1) # FV_dead
    add_features(descriptor, 'constrained_feature_variables', undead_feature_variables.difference(core_feature_variables)) # FV_constrained
    add_features(descriptor, 'undead_feature_variables', undead_feature_variables) # FV_undead
    add_features(descriptor, 'all_feature_variables', all_feature_variables) # FV
    add_features(descriptor, 'ALL_feature_variables', feature_variables.union(unconstrained_feature_variables)) # FV_all
    add_features(descriptor, 'features', features) # F
    add_features(descriptor, 'core_features', core_features, min=1)
    add_features(descriptor, 'unconstrained_features', unconstrained_features, min=1)
    add_features(descriptor, 'constrained_features', constrained_features)
    add_features(descriptor, 'added_features', added_features, min=0)
    add_features(descriptor, 'removed_features', removed_features, min=0)
    if extractor == 'kmax':
        potential_misses_grep.update([f for f in all_feature_variables.difference(features) if '__CONFIG_' not in f])
    return descriptor, feature_variables.union(unconstrained_feature_variables), features

def inspect_architecture_features_for_revision(extractor, revision, features_for_last_revision):
    config_features = set(df_configs[df_configs['revision'] == revision]['config'])
    architectures = [re.search('\[(.*)\]', f).group(1) for f in glob.glob(f'{output_directory}/kconfig/{extractor}/linux/{revision}[*.features')]
    architectures = list(set(architectures))
    architectures.sort()
    data = []
    total_features = set()
    total_feature_variables = set()
    features_for_current_revision = {}
    for architecture in architectures:
        descriptor, feature_variables, features = inspect_architecture_features_for_model(extractor, revision, architecture, config_features, features_for_last_revision)
        data.append(descriptor)
        total_features.update(features)
        features_for_current_revision[architecture] = features
        if extractor == 'kmax':
            total_feature_variables.update(feature_variables)
    for descriptor in data:
        add_features(descriptor, 'total_features', total_features)
        total_added_features = None
        total_removed_features = None
        if 'TOTAL' in features_for_last_revision and len(features_for_last_revision['TOTAL']) > 0:
            total_added_features = total_features.difference(features_for_last_revision['TOTAL'])
            total_removed_features = features_for_last_revision['TOTAL'].difference(total_features)
        add_features(descriptor, 'total_added_features', total_added_features, min=0)
        add_features(descriptor, 'total_removed_features', total_removed_features, min=0)
    features_for_current_revision['TOTAL'] = total_features
    df_configs_configurable.loc[(df_configs_configurable['revision'] == revision) & (df_configs_configurable['config'].isin(total_features)), 'configurable'] = True
    if extractor == 'kmax':
        potential_misses_kmax.update([f for f in config_features.difference(total_feature_variables)])
    return data, features_for_current_revision

def inspect_architecture_features(extractor):
    print(f'{extractor} ', end='')
    revisions = [re.search('/linux/(.*)\[', f).group(1) for f in glob.glob(f'{output_directory}/kconfig/{extractor}/linux/*.features')]
    revisions = list(set(revisions))
    revisions.sort(key=Version)
    data = []
    features_for_last_revision = {}
    i = 0
    for revision in revisions:
        i += 1
        if i % 10 == 0:
            print(revision + ' . ', end='')
        new_data, features_for_last_revision = inspect_architecture_features_for_revision(extractor, revision, features_for_last_revision)
        data += new_data
    print()
    return data

if os.path.isfile(f'{output_directory}/linux-features.dat'):
    with open(f'{output_directory}/linux-features.dat', 'rb') as f:
        [features_by_kind_per_architecture, df_extractor_comparison, potential_misses_grep, potential_misses_kmax, df_configs_configurable] = pickle.load(f)
else:
    features_by_kind_per_architecture = inspect_architecture_features('kconfigreader')
    features_by_kind_per_architecture += inspect_architecture_features('kmax')
    features_by_kind_per_architecture = pd.DataFrame(features_by_kind_per_architecture)
    df_extractor_comparison = []
    for key, value in extractor_comparison.items():
        [revision, architecture] = key.split('###')
        if type(value) is set:
            value = pd.NA
        df_extractor_comparison.append({'revision': revision, 'architecture': architecture, 'extractor_jaccard': value})
    df_extractor_comparison = pd.DataFrame(df_extractor_comparison)
    with open(f'{output_directory}/linux-features.dat', 'wb') as f:
        pickle.dump([features_by_kind_per_architecture, df_extractor_comparison, potential_misses_grep, potential_misses_kmax, df_configs_configurable], f)

replace_values(features_by_kind_per_architecture)
df_features = pd.merge(df_architectures, features_by_kind_per_architecture, how='outer').sort_values(by='committer_date')
df_features = pd.merge(df_kconfig, df_features, how='outer').sort_values(by='committer_date')

def compare_with_grep(message, list):
    print(f'{message}: ' + str(len(list)))
    print(pd.merge(df_configs[['config','kconfig-file']], pd.DataFrame(list, columns=['config']), how='inner') \
        .drop_duplicates().merge(df_config_types[['config', 'type']]).drop_duplicates())

def report_potential_misses(potential_misses_grep, potential_misses_kmax):
    # these are the features NOT found by grep, but found by kmax (this allows us to check whether the grep regex matches too much)
    # the only matches are enviroment variables (e.g., ARCH) and mistakes in kconfig files: IA64_SGI_UV (which has a trailing `) and SND_SOC_UX500_MACH_MOP500 (which has a leading +)
    compare_with_grep('#potential misses (grep)', potential_misses_grep)
    print()

    # these are the features found by grep, but NOT found by kmax, either constrained or unconstrained (this allows us to check whether kmax matches enough)
    # as there are some extraction failures for kmax, we expect some misses; also, we do not extract the um architecture; and finally, there are some test kconfig files that are never included
    # in the following, we try to filter out these effects (this is not perfect though)
    potential_misses_kmax_with_type = (pd.merge(df_configs[['config','kconfig-file', 'revision']], pd.DataFrame(potential_misses_kmax, columns=['config']), how='inner') \
            .drop_duplicates().merge(df_config_types[['config', 'type']]).drop_duplicates())
    misses_due_to_tests = set(potential_misses_kmax_with_type[ \
            potential_misses_kmax_with_type['kconfig-file'].str.startswith('Documentation/') | \
            potential_misses_kmax_with_type['kconfig-file'].str.startswith('scripts/')]['config'].unique())
    missing_kmax_models = df_features[(df_features['extractor'] == 'KClause') & df_features['#extracted_features'].isna()]
    missing_kmax_models = missing_kmax_models[['revision', 'architecture']].drop_duplicates()
    potential_misses_kmax_with_type['architecture'] = potential_misses_kmax_with_type['kconfig-file'].apply(lambda s: re.sub(r'^arch/(.*?)/.*$', r'\1', s))
    potential_misses_due_to_missing_kmax_models = set(potential_misses_kmax_with_type.merge(missing_kmax_models[['revision', 'architecture']].drop_duplicates()) \
                                                    .drop(columns=['kconfig-file', 'revision', 'architecture', 'type'])['config'].unique())
    potential_misses_kmax = potential_misses_kmax.difference(misses_due_to_tests).difference(potential_misses_due_to_missing_kmax_models)
    # the remaining matches are due to our way of using kmax extractor, where we ignore lines with new kconfig constructs like $(success,...)
    compare_with_grep('#potential misses (kmax)', potential_misses_kmax)

report_potential_misses(potential_misses_grep, potential_misses_kmax)

#potential misses (grep): 7
                        config             kconfig-file      type
0                         ARCH             init/Kconfig    string
53               KERNELVERSION             init/Kconfig    string
106                IA64_SGI_UV        arch/ia64/Kconfig      bool
187  SND_SOC_UX500_MACH_MOP500  sound/soc/ux500/Kconfig  tristate

#potential misses (kmax): 21
                                config                        kconfig-file  \
0                      MIPS_FPE_MODULE                 arch/mips64/Kconfig   
29                      BLK_DEV_FD1772         drivers/acorn/block/Kconfig   
83                         BLK_DEV_MFM         drivers/acorn/block/Kconfig   
137             BLK_DEV_MFM_AUTODETECT         drivers/acorn/block/Kconfig   
191                      VIRTEX_II_PRO      arch/ppc/platforms/4xx/Kconfig   
207                      VIRTEX_II_PRO  arch/powerpc/platforms/4xx/Kconfig   
223                          DRAM_BASE              arch/arm/Kconf

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_kconfig['committer_date'], y=df_kconfig['source_lines_of_code'], mode='markers', name='Source Lines of Code (r=.98)', marker_symbol='circle'),
    secondary_y=False
)

df_total_features = df_features.groupby(['extractor', 'revision']).agg({'#total_features': 'min'}).reset_index()
df_total_features = pd.merge(df_kconfig[['committer_date', 'revision']].drop_duplicates(), df_total_features)
df_total_features = df_total_features[df_total_features['extractor']=='KClause']
df = df_total_features.sort_values(by='committer_date')
df = df[['committer_date', '#total_features']].drop_duplicates()
fig.add_trace(
    go.Scatter(x=df['committer_date'], y=df['#total_features'], mode='markers', name='Number of Features (r=.99)', marker_symbol='circle'),
    secondary_y=True
)

fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Source Lines of Code", dtick=10000000, range=[0, 32000000], secondary_y=False)
fig.update_yaxes(title_text="Number of Features", dtick=5000, range=[0, 22000], secondary_y=True)

style_scatter(fig, marker_size=3, legend_position='topleft')
fig.update_xaxes(range=["2002-01-01", "2025-01-01"])
show(fig, 'sloc')
show(fig, 'sloc', width=330, height=310)

In [7]:
# statistics in section 1

# show(px.scatter(
#     df_kconfig,
#     x='committer_date',
#     y='source_lines_of_code',
#     trendline='ols'
# ))

# show(px.scatter(
#     df,
#     x='committer_date',
#     y='#total_features',
#     trendline='ols'
# ))

# linear regression of source lines of code and number of features
print(scipy.stats.pearsonr(df_kconfig['committer_date'].astype(int) // 10 ** 9, df_kconfig['source_lines_of_code']))
print(scipy.stats.pearsonr(df['committer_date'].astype(int) // 10 ** 9, df['#total_features']))
print(scipy.stats.linregress(df_kconfig['committer_date'].astype(int) // 10 ** 9, df_kconfig['source_lines_of_code']))
print(scipy.stats.linregress(df['committer_date'].astype(int) // 10 ** 9, df['#total_features']))

# lines of code per year
print(0.032409366566693104*60*60*24*365.25)
# features per year
print(2.6766885381596744e-05*60*60*24*365.25)


PearsonRResult(statistic=0.9811934407897785, pvalue=0.0)
PearsonRResult(statistic=0.9930302141366611, pvalue=9.164144370489585e-134)
LinregressResult(slope=0.032409366566693104, intercept=-31764431.646730997, rvalue=0.9811934407897667, pvalue=0.0, stderr=7.838559727898174e-05, intercept_stderr=106343.3916921083)
LinregressResult(slope=2.6766885381596744e-05, intercept=-24700.168680386334, rvalue=0.9930302141366613, pvalue=9.16414437047902e-134, stderr=2.6659859931417343e-07, intercept_stderr=357.78277809449696)
1022761.8263650743
844.6986621182774
